# Imports

In [ ]:
import os
import sys
sys.path.append('/home/rcendre/classification')

import itertools
import pandas
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from misvm import SIL, MISVM, NSK
from numpy import array, logspace
from scipy.stats import randint,uniform
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.models.models import CustomMIL, MultimodalClassifier
from toolbox.models.builtin import Applications
from toolbox.IO import dermatology
from sklearn.metrics import f1_score
from toolbox.transforms.common import PredictorTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [ ]:
# Advanced parameters
data_type='Full'
prefix = 'Sequential_Isotonic'
validation = 10
settings = Settings.get_default_dermatology()

# Photography

In [ ]:
p_inputs = IO.load('Photography.pickle')

In [ ]:
extractor = 'ResNetAvg'

# SVM Linear
model = Pipeline([('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
model_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [ ]:
low_folds = Tools.generate_folds(([1, 2, 3, 4, 5, 6, 7, 8], [9]), validation)
Tools.evaluate(p_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=low_folds, distribution=model_params, calibrate='isotonic') 

In [ ]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(p_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Photography'])))

# Dermoscopy

In [ ]:
d_inputs = IO.load('Dermoscopy.pickle')

In [ ]:
extractor = 'ResNetAvg'

# SVM Linear
model = Pipeline([('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
model_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [ ]:
low_folds = Tools.generate_folds(([1, 2, 3, 4, 5, 6, 7, 8], [9]), validation)
Tools.evaluate(d_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=low_folds, distribution=model_params, calibrate='isotonic') 

In [ ]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(d_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Dermoscopy'])))

# Microscopy

In [ ]:
m_inputs = IO.load('Microscopy.pickle')

In [ ]:
extractor = 'ResNetAvg'
all_image = [True] * len(m_inputs.index)
single_image = m_inputs['ID_Image'] == '0M'
Data.build_bags(m_inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', extractor)
m_inputs = m_inputs[single_image].reset_index()

In [ ]:
low_folds = Tools.generate_folds(([1, 2, 3, 4, 5, 6, 7, 8], [9]), validation)
model = CustomMIL(NSK(kernel='linear', C=1.0, verbose=False))
model_params = {'estimator__C': logspace(-2, 3, 6).tolist()}
Tools.evaluate(m_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=low_folds, distribution=model_params)

In [ ]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(m_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy'])))

# Fusion

In [ ]:
inputs = pandas.concat([p_inputs, d_inputs, m_inputs], axis=0)
inputs = inputs.reset_index(drop=True)
 # Save
IO.save(inputs, f'Low_{prefix}.pickle')

# Sequential

In [ ]:
inputs = IO.load(f'Low_{prefix}.pickle')

In [ ]:
all_image = [True] * len(inputs.index)
single_image = inputs['Modality'] == 'Photography'
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'Prediction_{Tools.PROBABILITY}')
inputs = inputs[single_image].reset_index()

In [ ]:
high_folds = Tools.generate_folds(([1], [2]), validation)
modality = MultimodalClassifier(method='modality', metric=f1_score)
modality_rev = MultimodalClassifier(method='modality', metric=f1_score, ordered=False)
classe = MultimodalClassifier(method='modality_class', metric=f1_score)
classe_rev = MultimodalClassifier(method='modality_class', metric=f1_score, ordered=False)

modality_ones = MultimodalClassifier(method='modality', metric=f1_score, from_zero=False)
modality_rev_ones = MultimodalClassifier(method='modality', metric=f1_score, ordered=False, from_zero=False)
classe_ones = MultimodalClassifier(method='modality_class', metric=f1_score, from_zero=False)
classe_rev_ones = MultimodalClassifier(method='modality_class', metric=f1_score, ordered=False, from_zero=False)

In [ ]:
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, modality, 'modality')
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, modality_rev, 'modality_rev')
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, classe, 'modality_class') 
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, classe_rev, 'modality_class_rev')

Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, modality, 'modality_ones')
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, modality_rev, 'modality_rev_ones')
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, classe, 'modality_class_ones') 
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, classe_rev, 'modality_class_rev_ones') 
IO.save(inputs, f'High_{prefix}.pickle')

# Score

In [ ]:
from IPython.display import HTML
from IPython.display import display

prefix = 'Sequential_Isotonic'
diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'modality'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()

name = f'modality_rev'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()

name = f'modality_class'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()

name = f'modality_class_rev'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()

name = f'modality_ones'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()

name = f'modality_rev_ones'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()

name = f'modality_class_ones'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()

name = f'modality_class_rev_ones'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))
figure = Views.steps_visualization(inputs, {'label_encode': 'LesionEncode', 'eval': name})
figure.savefig(f'{prefix}_{name}.svg')
figure.show()